In [71]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_name = 'republic-project'
repo_dir = os.path.split(os.getcwd())[0].split(repo_name)[0] + repo_name
print("adding project dir to path:", repo_dir)
if repo_dir not in sys.path:
    sys.path = [repo_dir] + sys.path
else:
    sys.path.remove(repo_dir)
    sys.path = [repo_dir] + sys.path
    


adding project dir to path: /Users/marijnkoolen/Code/Huygens/republic-project


## Reading the IOB-formatted files

In [72]:
import glob
import os
from collections import defaultdict

# this is the folder in which train, test and dev files will be saved
data_folder = f'{repo_dir}/ground_truth/entities/tag_de_besluiten/tdb-data'
train_folder = f'{repo_dir}/ground_truth/entities/tag_de_besluiten/flair_training'

layer_files = defaultdict(list)

tagged_res_files = glob.glob(os.path.join(data_folder, 'session**/*.tsv'))
for tagged_res_file in tagged_res_files:
    fname_parts = tagged_res_file.split('/')
    layer_name = fname_parts[-1].replace('.tsv', '')
    res_id = fname_parts[-2].replace('.txt', '')
    if layer_name == 'all_tags':
        continue
    else:
        layer_name = layer_name.replace('tag_', '')
    layer_files[layer_name].append(tagged_res_file)


## Creating the Train, Validate and Test files

Randomly distributing the content of the IOB-formatted files, using an 80-10-10 split.

In [73]:
[i / 10 for i in range(1, 10)]

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [74]:
import random

import numpy as np


def pick_output_filehandles(fhs):
    pick_set = random.randint(0, 9)
    output_fhs = []
    if pick_set < 8:
        pick_subset = random.random()
        for i in [i / 10 for i in range(1, 11)]:
            if pick_subset <= i:
                output_fhs.append(fhs[f'train_{i}'])
    elif pick_set == 8:
        output_fhs.append(fhs['validate'])
    else:
        output_fhs.append(fhs['test'])
    return output_fhs


# pick a random seed so we can repeat the process
random_seed = 89234
random.seed(random_seed)

    
for layer_name in layer_files:
    layer_dir = f'{repo_dir}/ground_truth/entities/tag_de_besluiten/flair_training_{layer_name}'
    if os.path.exists(layer_dir) is False:
        os.mkdir(layer_dir)
    train_file = os.path.join(layer_dir, 'train.txt')
    validate_file = os.path.join(layer_dir, 'validate.txt')
    test_file = os.path.join(layer_dir, 'test.txt')
    fhs = {}
    for i in [i / 10 for i in range(1, 11)]:
        fhs[f"train_{i}"] = open(os.path.join(layer_dir, f'train_{i}.txt'), 'wt')
    #fh_train = open(train_file, 'wt')
    fhs['validate'] = open(validate_file, 'wt')
    fhs['test'] = open(test_file, 'wt')
    for layer_file in layer_files[layer_name]:
        #out_fh = pick_output_filehandle([fh_train, fh_validate, fh_test])
        out_fhs = pick_output_filehandles(fhs)
        with open(layer_file, 'rt') as fh_in:
            for line in fh_in:
                for out_fh in out_fhs:
                    out_fh.write(line.replace('\t', ' '))
            for out_fh in out_fhs:
                out_fh.write('\n')
    fh_train.close()
    fh_validate.close()
    fh_test.close()


In [58]:
import torch

torch.backends.mps.is_available()
torch.device("mps")

device(type='mps')